In [1]:
##Libraries
#Dataframe & Arrays
import pandas as pd
import numpy as np



In [2]:
df = pd.read_csv('../data/dataframe_awp_staging.csv')

In [3]:
df = df.sort_values(by=['StockCode','UnitPrice'], ascending=False)

In [4]:
df_pricing_file = df.groupby(['B2B','StockCode','UnitPrice']).agg({'Rev': ['sum','count'],'Quantity':['sum']}).reset_index()
df_pricing_file = df_pricing_file.sort_values(by=['StockCode','UnitPrice'], ascending=False)
df_pricing_file.head()

B2B StockCode UnitPrice    Rev       Quantity
                                 sum count      sum
9105  1.0    90214Z      0.29   3.48     1       12
9104  1.0    90214Y      1.25   1.25     1        1
9103  1.0    90214Y      0.29  17.40     2       60
9102  1.0    90214W      0.29   3.48     1       12
9101  1.0    90214V      1.25   2.50     2        2

In [5]:
df_pricing_file.columns = ['B2B','StockCode','UnitPrice','Rev','Count','Qty']
df_pricing_file.head()

,B2B,StockCode,UnitPrice,Rev,Count,Qty
9105,1.0,90214Z,0.29,3.48,1,12
9104,1.0,90214Y,1.25,1.25,1,1
9103,1.0,90214Y,0.29,17.40,2,60
9102,1.0,90214W,0.29,3.48,1,12
9101,1.0,90214V,1.25,2.50,2,2


In [6]:
df_awp = df_pricing_file.groupby(['B2B','StockCode']).agg({'Rev': ['sum'],'Qty':['sum']})
df_awp.columns = ['Rev','Qty']

In [7]:
df_awp.head()

Rev  Qty
B2B StockCode             
0.0 11001        1.69    1
    15036      369.96  612
    15039       18.70   22
    15044A       5.90    2
    15044B       8.85    3

In [8]:
def AWP(row):
    return (row['Rev'] / row['Qty'])

In [9]:
df_awp['AWP'] = (df_awp['Rev'] / df_awp['Qty'])
df_awp.head()

Rev  Qty      AWP
B2B StockCode                      
0.0 11001        1.69    1  1.69000
    15036      369.96  612  0.60451
    15039       18.70   22  0.85000
    15044A       5.90    2  2.95000
    15044B       8.85    3  2.95000

In [10]:
df_awp.reset_index(inplace=True)
df_awp.columns  = ['B2B','StockCode','Rev','Qty','AWP']
df_awp.head()

,B2B,StockCode,Rev,Qty,AWP
0,0.0,11001,1.69,1,1.69000
1,0.0,15036,369.96,612,0.60451
2,0.0,15039,18.70,22,0.85000
3,0.0,15044A,5.90,2,2.95000
4,0.0,15044B,8.85,3,2.95000


In [11]:
## trying to identify average  Qty purchased by StockCode and customer type

In [12]:
df_qty_mean = df.groupby(['B2B','StockCode'])['Quantity'].mean().astype(int).to_frame()
df_qty_mean.reset_index(inplace=True)
df_qty_mean.columns  = ['B2B','StockCode','Quantity']

In [13]:
df_qty_mean.head()

,B2B,StockCode,Quantity
0,0.0,11001,1
1,0.0,15036,153
2,0.0,15039,11
3,0.0,15044A,2
4,0.0,15044B,1


In [14]:
def qtymeanlookup(row):
    """
    Input: StockCode taken from row within dataframe
    Output: The corresponding mean quantity sold for that item
    
    The purpose of this funtion is to vlookup value of item from a reference dataframe (df_qty_mean)
    """
    try:

        return int(df_qty_mean.loc[(df_qty_mean['StockCode'] == row.StockCode) & (df_qty_mean['B2B'] == row.B2B)]["Quantity"])

    except:

        pass

In [18]:
df_awp['MeanQty'] = df_awp.apply(qtymeanlookup, axis=1)

In [19]:
df_awp.head()

,B2B,StockCode,Rev,Qty,AWP,ModeQty,MeanQty
0,0.0,11001,1.69,1,1.69000,1,1
1,0.0,15036,369.96,612,0.60451,153,153
2,0.0,15039,18.70,22,0.85000,11,11
3,0.0,15044A,5.90,2,2.95000,2,2
4,0.0,15044B,8.85,3,2.95000,1,1


In [20]:
df_awp.to_csv('../data/df_awp.csv', index=False)